# 利用 Bert 计算 相似度

## 启动服务

In [ ]:
```
    PS D:\project\python_wp\nlp\data\chinese_wwm_ext_L-12_H-768_A-12> bert-serving-start -model_dir ./ -num_worker=2
```

## 导入库

In [2]:
import os
from gensim import corpora, models, similarities
from pprint import pprint
from matplotlib import pyplot as plt
import jieba
import numpy as np
jieba.load_userdict("../../data/dict.txt")
np.random.seed(1) # 设置随即种子数，以便相同的设置能跑出相同的结果，可复现
from bert_serving.client import BertClient
bc = BertClient()

## 函数编写

In [20]:
def cosine(a,b):
    return a.dot(b)/(np.linalg.norm(a)*np.linalg.norm(b))

## 加载数据

In [3]:
doc_path = "THUCNews/1.txt"
documents = [line.strip().split("\t")[0] for line in open(doc_path,encoding="utf-8").readlines() ]
len(documents),documents[0:2]

(120,
 ['中国大陆方面，股市迎来一波猛涨，站上五年来的最高点。本周，因新冠疫情延期一月的中国高考拉开帷幕，但高考前多起包括冒名顶替在内的舞弊案件引发对公平性的讨论。北京新一轮疫情得到控制，本周首现零新增。同样在北京，知名法学家许章润被警察带走，最近几年他发表多篇文章批评中国当局。',
  '香港《国安法》在香港和海外持续发酵。本周BBC中文通过五张图为你详解其中的法律重点，比如相关机构组成和权力分配等。西方国家对香港《国安法》反应强烈，此前英国承诺为香港BNO护照持有者开启移民入籍路径，本周BBC中文采访探讨这一政策是否会导致香港移民潮。本周澳大利亚政府也推出一系列措施，作为回应，包括暂停与香港的引渡协议、更新旅游建议以及为香港居民提供签证便利等。中国则对两国的干涉表示反对，称澳方成天把反对"外国干涉"挂在嘴边，却在涉港问题上说三道四，充分暴露了其虚伪性和双重标准。'])

## 计算相似度

In [10]:
from termcolor import colored
topk = 10
query = "中国大陆方面，股市迎来一波猛涨，站上五年来的最高点。本周，因新冠疫情延期一月的中国高考拉开帷幕，但高考前多起包括冒名顶替在内的舞弊案件引发对公平性的讨论。北京新一轮疫情得到控制，本周首现零新增。同样在北京，知名法学家许章润被警察带走，最近几年他发表多篇文章批评中国当局。"

In [23]:
with BertClient(port=5555, port_out=5556) as bc:      # 获取句子向量编码   
    query_vec = bc.encode([query])[0]
    doc_vecs = bc.encode(documents) 
    # 余弦相似度 分数计算。         
    # np.linalg.norm 是求取向量的二范数，也就是向量长度。
    score_list = []
    for doc_vec in doc_vecs:
        score_list.append(cosine(query_vec, doc_vec) )
    '''         		
    argsort()函数是将x中的元素从小到大排列，提取其对应的index(索引)                     
    [::-1]取从后向前（相反）的元素, 例如[ 1 2 3 4 5 ]            
    则输出为[ 5 4 3 2 1 ]         
    '''         
    topk_idx = np.argsort(score_list)[::-1][:topk]         
    print('top %d questions similar to "%s"' % (topk, colored(query, 'green')))         
    for idx in topk_idx:             
        print('> %s\t%s\t%s' % (colored('%.5f' % score_list[idx], 'cyan'),idx, colored(documents[idx], 'yellow')))

top 10 questions similar to "中国大陆方面，股市迎来一波猛涨，站上五年来的最高点。本周，因新冠疫情延期一月的中国高考拉开帷幕，但高考前多起包括冒名顶替在内的舞弊案件引发对公平性的讨论。北京新一轮疫情得到控制，本周首现零新增。同样在北京，知名法学家许章润被警察带走，最近几年他发表多篇文章批评中国当局。"
> 1.00000	0	中国大陆方面，股市迎来一波猛涨，站上五年来的最高点。本周，因新冠疫情延期一月的中国高考拉开帷幕，但高考前多起包括冒名顶替在内的舞弊案件引发对公平性的讨论。北京新一轮疫情得到控制，本周首现零新增。同样在北京，知名法学家许章润被警察带走，最近几年他发表多篇文章批评中国当局。
> 0.90828	2	新冠疫情还没完全消退，中国股市在7月初则迎来一轮暴涨，港股也几乎同步放量上涨。截止本周三（7月8日），中国股市沪深300指数已连续七个交易日上涨。七天内，沪综指连续突破3000点、3100点、3200点、3300点和3400点五个整数关口。创下五年来高位。市场建立起对经济复苏的预期，促使本轮股市上涨。但也有声音认为，目前经济状况并不支持这种程度的暴涨，恐重蹈2015年“股灾”覆辙。本周BBC也报道了疫情影响下面临困境的中国底层农民工和大学毕业生。
> 0.86305	1	香港《国安法》在香港和海外持续发酵。本周BBC中文通过五张图为你详解其中的法律重点，比如相关机构组成和权力分配等。西方国家对香港《国安法》反应强烈，此前英国承诺为香港BNO护照持有者开启移民入籍路径，本周BBC中文采访探讨这一政策是否会导致香港移民潮。本周澳大利亚政府也推出一系列措施，作为回应，包括暂停与香港的引渡协议、更新旅游建议以及为香港居民提供签证便利等。中国则对两国的干涉表示反对，称澳方成天把反对"外国干涉"挂在嘴边，却在涉港问题上说三道四，充分暴露了其虚伪性和双重标准。
> 0.85480	47	据普帕尔表示，中国电信巨擘不会遭到“完全禁用”，但采用华为的法国电信业者将会有最长8年的授权许可。“不采用华为的电信业者，我们鼓励他们不要这么做。至于已经使用的，我们会核发3年至8年的授权许可”。
> 0.85287	71	7月1日，印度和中国完成第三轮军长级会谈，虽然有新闻报道，双方还在不断向边境增兵，以应对冲突升级，但像6月15日那样的冲突终归还未发生，